# IPAI Individual Challenge Notebook - Data Cleaning Expert

Written by: Nuno Correia, 58638 Group 10

The dataset selected for this individual challenge was: Air Pollution in the US.

# Imports

In [221]:
import pandas as pd
import numpy as np
import random

pollution_original = pd.read_csv("pollution_2000_2023.csv", index_col=[0])

In [222]:
pollution_copy = pollution_original.copy()

In [223]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [224]:
%pip install nlpaug

Note: you may need to restart the kernel to use updated packages.


# Creating complex data cleaning challenges

The library I'll be using to insert complex errors in the strings is nplaug.

In [225]:
import nlpaug.augmenter.char as nac

The functions that will be used do the following:
* Create common keyboard typos in State, County and City strings with a fixed probability;
* Introduce NaNs with a fixed probability;

There is a 50% chance that State, County and City get common keyboard typos and there is a 30% chance that any column except ,Date and Address, get a value dropped.

In [226]:
pollution_copy.State = pollution_copy.State.apply(
                                        lambda x: nac.KeyboardAug(aug_char_p=0.2).augment(x)[0] 
                                                if random.random() > 0.5 and isinstance(x, str) 
                                                else x 
                                        )
pollution_copy.County = pollution_copy.County.apply(
                                        lambda x: nac.KeyboardAug(aug_char_p=0.2).augment(x)[0] 
                                                if random.random() > 0.5 and isinstance(x, str) 
                                                else x 
                                        )
pollution_copy.City = pollution_copy.City.apply(
                                        lambda x: nac.KeyboardAug(aug_char_p=0.2).augment(x)[0] 
                                                if random.random() > 0.5 and isinstance(x, str) 
                                                else x 
                                        )

In [227]:
def insert_nan(data, column ,prob=0.2):
    for index in data.index:
        if random.random() < prob:
            data.at[index, column] = np.nan

for column in pollution_copy.columns[5:]:
    insert_nan(pollution_copy, column)

The result of the execution of these operations is below:

In [228]:
pollution_copy.isnull().sum()

Date                      0
Address                   0
State                     0
County                    0
City                      0
O3 Mean              133484
O3 1st Max Value     132991
O3 1st Max Hour      132906
O3 AQI               133324
CO Mean              133499
CO 1st Max Value     133035
CO 1st Max Hour      132338
CO AQI               132879
SO2 Mean             132758
SO2 1st Max Value    133564
SO2 1st Max Hour     133073
SO2 AQI              133334
NO2 Mean             133035
NO2 1st Max Value    132653
NO2 1st Max Hour     133712
NO2 AQI              132688
dtype: int64

# State of the original dataset

A quick check on the first 5 columns in the dataset.

In [229]:
pollution_original.head()

Date                                  Address    State    County  \
0  2000-01-01  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa   
1  2000-01-02  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa   
2  2000-01-03  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa   
3  2000-01-04  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa   
4  2000-01-05  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  Arizona  Maricopa   

      City   O3 Mean  O3 1st Max Value  O3 1st Max Hour  O3 AQI   CO Mean  \
0  Phoenix  0.019765             0.040               10      37  0.878947   
1  Phoenix  0.015882             0.032               10      30  1.066667   
2  Phoenix  0.009353             0.016                9      15  1.762500   
3  Phoenix  0.015882             0.033                9      31  1.829167   
4  Phoenix  0.007353             0.012                9      11  2.700000   

   ...  CO 1st Max Hour  CO AQI  SO2 Mean  SO2 1st Max Value  \
0  ...               23    25.0  3.000000                9.0   
1  ...                0    26.0  1.958333                3.0   
2  ...                8    28.0  5.250000               11.0   
3  ...               23    34.0  7.083333               16.0   
4  ...                2    42.0  8.708333               15.0   

   SO2 1st Max Hour  SO2 AQI   NO2 Mean  NO2 1st Max Value  NO2 1st Max Hour  \
0                21     13.0  19.041667               49.0                19   
1                22      4.0  22.958333               36.0                19   
2                19     16.0  38.125000               51.0                 8   
3                 8     23.0  40.260870               74.0                 8   
4                 7     21.0  48.450000               61.0                22   

   NO2 AQI  
0       46  
1       34  
2       48  
3       72  
4       58  

[5 rows x 21 columns]

The dataset has the following columns:

In [230]:
pollution_original.columns

Index(['Date', 'Address', 'State', 'County', 'City', 'O3 Mean',
       'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'CO Mean',
       'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI', 'SO2 Mean',
       'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI', 'NO2 Mean',
       'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI'],
      dtype='object')

# Data cleaning the dirty dataset

## Cleaning the State column

To begin cleaning the State column, I'll be using an array that contains all possible states in the US. 

In [231]:
us_states = [
    "alabama", "alaska", "arizona", "arkansas", "california", "colorado",
    "connecticut", "delaware", "florida", "georgia", "hawaii", "idaho",
    "illinois", "indiana", "iowa", "kansas", "kentucky", "louisiana",
    "maine", "maryland", "massachusetts", "michigan", "minnesota",
    "mississippi", "missouri", "montana", "nebraska", "nevada",
    "new hampshire", "new jersey", "new mexico", "new york",
    "north carolina", "north dakota", "ohio", "oklahoma", "oregon",
    "pennsylvania", "rhode island", "south carolina", "south dakota",
    "tennessee", "texas", "utah", "vermont", "virginia", "washington",
    "west virginia", "wisconsin", "wyoming"
]

Next, I'm going to use 2 similarity measures to find similarity matches between the states with typos and the correct spelling. These two similarity measures are:
* Levenshtein: Perfect to capture small differences between two strings, since we're dealing with typos this similarity measures fits the problem quite well;
* JaroWinkler: Good to match prefixes in strings, this similarity measure will be useful when the beginning of a string isn't changes much or at all;

In [232]:
import py_stringmatching
import re

pollution_copy.State = pollution_copy.State.str.lower()
pollution_copy.State = pollution_copy.State.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

def clean_state(state):
    best_score = 0
    best_state = None
    leven = py_stringmatching.Levenshtein()
    jw = py_stringmatching.JaroWinkler()
    if pd.isna(state):
        return state
    for s in us_states:
        leven_score = leven.get_sim_score(state, s)
        jaro_score = jw.get_sim_score(state, s)
        score = leven_score * 0.5 + jaro_score * 0.5

        if score == 1:
            return s
        
        if score > best_score:
            best_score = score
            best_state = s

    return best_state

pollution_copy.State = pollution_copy.State.apply(lambda x: clean_state(x) if isinstance(x, str) else x)
pollution_copy.State = pollution_copy.State.str.replace(" ", "")

By using the function above, we get the cleaned versions of the states, and these are mostly correct.

In [233]:
pollution_copy.State.unique()

array(['arizona', 'california', 'colorado', 'southcarolina', 'missouri',
       'wisconsin', 'northcarolina', 'connecticut', 'westvirginia',
       'mississippi', 'virginia', 'florida', 'illinois', 'indiana',
       'kansas', 'kentucky', 'louisiana', 'michigan', 'newjersey',
       'newyork', 'oklahoma', 'pennsylvania', 'texas', 'washington',
       'southdakota', 'massachusetts', 'nevada', 'newhampshire',
       'tennessee', 'nebraska', 'georgia', 'northdakota', 'iowa', 'maine',
       'maryland', 'arkansas', 'oregon', 'wyoming', 'idaho', 'ohio',
       'delaware', 'hawaii', 'minnesota', 'newmexico', 'rhodeisland',
       'utah', 'alabama', 'alaska', 'vermont'], dtype=object)

## Cleaning the County column

To begin I'll remove all the whitespaaces and make all the strings lower case for normalization's sake.

In [234]:
pollution_copy.County = pollution_copy.County.str.lower()
pollution_copy.County = pollution_copy.County.str.strip()

To clean the counties, there's a .txt file found at (https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt) that contains all the counties for a given state. Therefore, my methodology to clean these will be to find matches between the correct spelling of counties for a specific state.

In [235]:
cols = ['State', 'State_FIPS', 'County_FIPS', 'County_Name', 'Class_Code']
counties_list = pd.read_csv("national_county.txt", header=None, names=cols, dtype=str)
counties_per_state = counties_list[["State", "County_Name"]].copy()

However, this .txt requires some data cleaning itself. The procedures used to clean and format the data to a convinient format for the pollution_copy dataset, I'll apply the following transformations:
* Changing the State abbreviations to full state names;
* Removing spaces from the County_Name column;
* Making the County_Name column all lower;
* Stripping the County_Name column of whitespace;
* Removing references to some common county designations which do not show up on the pollution_copy dataset;
* Removing the spaces from the County_Name column;

In [236]:
us_states = {
    'AL': 'alabama',
    'AK': 'alaska',
    'AZ': 'arizona',
    'AR': 'arkansas',
    'CA': 'california',
    'CO': 'colorado',
    'CT': 'connecticut',
    'DE': 'delaware',
    'FL': 'florida',
    'GA': 'georgia',
    'HI': 'hawaii',
    'ID': 'idaho',
    'IL': 'illinois',
    'IN': 'indiana',
    'IA': 'iowa',
    'KS': 'kansas',
    'KY': 'kentucky',
    'LA': 'louisiana',
    'ME': 'maine',
    'MD': 'maryland',
    'MA': 'massachusetts',
    'MI': 'michigan',
    'MN': 'minnesota',
    'MS': 'mississippi',
    'MO': 'missouri',
    'MT': 'montana',
    'NE': 'nebraska',
    'NV': 'nevada',
    'NH': 'new hampshire',
    'NJ': 'new jersey',
    'NM': 'new mexico',
    'NY': 'new york',
    'NC': 'north carolina',
    'ND': 'north dakota',
    'OH': 'ohio',
    'OK': 'oklahoma',
    'OR': 'oregon',
    'PA': 'pennsylvania',
    'RI': 'rhode island',
    'SC': 'south carolina',
    'SD': 'south dakota',
    'TN': 'tennessee',
    'TX': 'texas',
    'UT': 'utah',
    'VT': 'vermont',
    'VA': 'virginia',
    'WA': 'washington',
    'WV': 'west virginia',
    'WI': 'wisconsin',
    'WY': 'wyoming',
    'DC': 'district of columbia',
    'AS': 'american samoa',
    'GU': 'guam',
    'MP': 'northern mariana islands',
    'PR': 'puerto rico',
    'VI': 'virgin islands',
    'UM': 'united states minor outlying islands'
}

counties_per_state.State = counties_per_state.State.apply(lambda x: us_states[x] if isinstance(x, str) else x)
counties_per_state.State = counties_per_state.State.str.replace(" ", "")

In [237]:
counties_per_state.County_Name = counties_per_state.County_Name.str.lower()
counties_per_state.County_Name = counties_per_state.County_Name.str.strip()
counties_per_state.County_Name = counties_per_state.County_Name.apply(lambda x: re.sub(r'\s+(county|parish|borough|census area|municipality)$', '', x))
counties_per_state.County_Name = counties_per_state.County_Name.str.replace(" ", "")

The following function will be used to clean the County column, this function does the following:
* It uses the State column, if it's not NaN, to reduce the number of comparisons when finding similarity matches between the correct spelling of county names;
* It compares all the correct spellings of counties in a state with the incorrect county spelling to find the most similar one, using the same method as the State function above;

In [238]:
def clean_county(state, county):
    best_score = 0
    best_county = None
    leven = py_stringmatching.Levenshtein()
    jw = py_stringmatching.JaroWinkler()
    
    if pd.isna(county):
        return county
    if pd.isna(state):
        return county
    
    for c in counties_per_state.query("State == '{0}'".format(state)).County_Name.values:
        leven_score = leven.get_sim_score(county, c)
        jaro_score = jw.get_sim_score(county, c)
        score = leven_score * 0.5 + jaro_score * 0.5

        if score == 1:
            return c
        
        if score > best_score:
            best_score = score
            best_county = c

    return best_county

In [239]:
pollution_copy["County"] = pollution_copy.apply(
    lambda row: clean_county(row["State"], row["County"]), axis=1
)

In [240]:
pollution_copy.head()

Date                                  Address    State    County  \
0  2000-01-01  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  arizona  maricopa   
1  2000-01-02  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  arizona  maricopa   
2  2000-01-03  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  arizona  maricopa   
3  2000-01-04  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  arizona  maricopa   
4  2000-01-05  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN  arizona  maricopa   

      City   O3 Mean  O3 1st Max Value  O3 1st Max Hour  O3 AQI   CO Mean  \
0  Pmosnix  0.019765               NaN             10.0    37.0  0.878947   
1  Pto3nix  0.015882               NaN             10.0     NaN  1.066667   
2  Phoenix  0.009353             0.016              9.0     NaN  1.762500   
3  Phoenix  0.015882               NaN              9.0    31.0  1.829167   
4  PhoWnia  0.007353             0.012              9.0     NaN       NaN   

   ...  CO 1st Max Hour  CO AQI  SO2 Mean  SO2 1st Max Value  \
0  ...             23.0    25.0  3.000000                9.0   
1  ...              0.0    26.0       NaN                NaN   
2  ...              8.0    28.0  5.250000               11.0   
3  ...             23.0    34.0  7.083333               16.0   
4  ...              2.0     NaN  8.708333               15.0   

   SO2 1st Max Hour  SO2 AQI   NO2 Mean  NO2 1st Max Value  NO2 1st Max Hour  \
0              21.0      NaN  19.041667               49.0              19.0   
1              22.0      NaN  22.958333               36.0              19.0   
2              19.0     16.0  38.125000               51.0               8.0   
3               8.0     23.0  40.260870               74.0               8.0   
4               7.0      NaN  48.450000                NaN              22.0   

   NO2 AQI  
0     46.0  
1     34.0  
2      NaN  
3     72.0  
4     58.0  

[5 rows x 21 columns]

## Cleaning the City column

Normalizing the strings in the City column.

In [241]:
pollution_copy.City = pollution_copy.City.str.lower()
pollution_copy.City = pollution_copy.City.str.strip()
pollution_copy.City = pollution_copy.City.str.replace(" ", "")
pollution_copy.City = pollution_copy.City.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

To data clean the City column, I'll be using a .csv that includes states and their counties, since I've already data cleaned both states and counties. This should help reduce the number of comparisons needed to correct typos in the City column.

The US cities and counties dataset can be found here, the free tier was used: https://simplemaps.com/data/us-cities

In [242]:
us_cities_info = pd.read_csv("uscities.csv")

In [243]:
us_cities_info = us_cities_info[["city", "state_name", "county_name", "zips"]].copy()

Since this dataset's strings are not normalized, I'll normalize them now. The following transformations to the columns city,state_name and county_name were done:
* Made all the strings lower case;
* Stripped all the whitespace from the strings;
* Removed all spaces from them;
* Removed all special chracters from the strings;

In [244]:
us_cities_info.city = us_cities_info.city.str.lower()
us_cities_info.city = us_cities_info.city.str.strip()
us_cities_info.city = us_cities_info.city.str.replace(" ", "")
us_cities_info.city = us_cities_info.city.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

In [245]:
us_cities_info.state_name = us_cities_info.state_name.str.lower()
us_cities_info.state_name = us_cities_info.state_name.str.strip()
us_cities_info.state_name = us_cities_info.state_name.str.replace(" ", "")
us_cities_info.state_name = us_cities_info.state_name.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

In [246]:
us_cities_info.county_name = us_cities_info.county_name.str.lower()
us_cities_info.county_name = us_cities_info.county_name.str.strip()
us_cities_info.county_name = us_cities_info.county_name.str.replace(" ", "")
us_cities_info.county_name = us_cities_info.county_name.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x)

After the running the code a couple of times, there are alsom some State-County-City combinations missing from the us_cities_info. The following will be added (State-County-City):
* Connecticut, Fairfield, Westport
* Virginia, Alexandria City, Alexandria
* Connecticut, Litchfield, Cornwall
* Connecticut, Hampton City, East Hartford
* Connecticut, Hampton City, Cornwall
* Connecticut, New Haven, New Haven
* Missouri, St. Louis City, St. Louis

I've also added "notinacity" as there are a couple readings in the Alexandria City County in Virgina that do not occur in a city.

In [247]:
us_cities_info.columns

Index(['city', 'state_name', 'county_name', 'zips'], dtype='object')

In [248]:
# st.louis, st.louiscity
# newhaven, newhaven
# hampton, hamptoncity
# easthartford, hartford
# cornwall, litchfield
# notinacity, alexandriacity
# alexandria, alexandriacity
# westport, fairfield

us_cities_info.loc[len(us_cities_info)] = ["st.louis", "missouri", "st.louiscity", None]
us_cities_info.loc[len(us_cities_info)] = ["newhaven", "connecticut", "newhaven", None]
us_cities_info.loc[len(us_cities_info)] = ["cornwall", "connecticut", "litchfield", None]
us_cities_info.loc[len(us_cities_info)] = ["easthartford", "connecticut", "hamptoncity", None]
us_cities_info.loc[len(us_cities_info)] = ["cornwall", "connecticut", "hamptoncity", None]
us_cities_info.loc[len(us_cities_info)] = ["westport", "connecticut", "fairfield", None]
us_cities_info.loc[len(us_cities_info)] = ["alexandria", "virginia", "alexandriacity", None]
us_cities_info.loc[len(us_cities_info)] = ["notinacity", "virginia", "alexandriacity", None]

Next I'll define the cleaning function to be used.

In [249]:
leven = py_stringmatching.Levenshtein()
jw = py_stringmatching.JaroWinkler()

def clean_city(state, county, city):
    best_score = 0
    best_city = city
    
    city_candidates = us_cities_info[(us_cities_info.state_name == state) & (us_cities_info.county_name == county)] if \
                      not pd.isna(state) else us_cities_info[us_cities_info.county_name == county]

    for city_candidate in city_candidates.city.values:
        leven_score = leven.get_sim_score(city, city_candidate)
        jaro_score = jw.get_sim_score(city, city_candidate)
        score = leven_score * 0.5 + jaro_score * 0.5

        if score == 1:
            return city_candidate
        
        if score > best_score:
            best_score = score
            best_city = city_candidate
            
    return best_city    

In [250]:
from tqdm import tqdm
tqdm.pandas()

pollution_copy["City"] = pollution_copy.progress_apply(
    lambda row: clean_city(row["State"], row["County"], row["City"]), axis=1
)

100%|██████████| 665414/665414 [28:59<00:00, 382.51it/s]


## Filling NaN values in the numerical columns

To obtain the missing numerical values, I created a loop that goes over all numerical values and groups them by address and the desired metric and then it computes the median of the previous values in a given place and it fills them.

In [251]:
tqdm.pandas()
for numerical_column in pollution_copy.columns[5:]:
    address_medians = pollution_copy.groupby("Address")[numerical_column].median()

    pollution_copy[numerical_column] = pollution_copy.progress_apply(
        lambda row: address_medians[row.Address] if pd.isna(row[numerical_column]) else row[numerical_column],
        axis=1
    )

100%|██████████| 665414/665414 [00:02<00:00, 224157.49it/s]


In [252]:
pollution_copy.isna().sum()

Date                 0
Address              0
State                0
County               0
City                 0
O3 Mean              0
O3 1st Max Value     0
O3 1st Max Hour      0
O3 AQI               0
CO Mean              0
CO 1st Max Value     0
CO 1st Max Hour      0
CO AQI               0
SO2 Mean             0
SO2 1st Max Value    0
SO2 1st Max Hour     0
SO2 AQI              0
NO2 Mean             0
NO2 1st Max Value    0
NO2 1st Max Hour     0
NO2 AQI              0
dtype: int64